In [1]:
from pandas import DataFrame
from pandas import concat
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()

    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [(df.columns[j] + '(t-{})'.format(i)) for j in range(n_vars)]

    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [(df.columns[j] + '(t)') for j in range(n_vars)]
        else:
            names += [(df.columns[j] + '(t+{})'.format(i)) for j in range(n_vars)]

    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names

    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)

    return agg

In [2]:
import pandas as pd

df = pd.read_csv("data.csv", index_col=0)
df.index = pd.to_datetime(df.index)
df = df.resample('1d').mean()
df.rename(columns={ df.columns[1]: "Temp_zewn" }, inplace=True)
df.rename(columns={ df.columns[2]: "V_wiatru" }, inplace=True)
df.rename(columns={ df.columns[3]: "Wilg" }, inplace=True)
df.rename(columns={ df.columns[5]: "Dlug_dnia" }, inplace=True)
df.rename(columns={ df.columns[6]: "Typ_dnia" }, inplace=True)
df.rename(columns={ df.columns[7]: "Pora_roku" }, inplace=True)
df.head()

,Energia,Temp_zewn,V_wiatru,Wilg,Zachm,Dlug_dnia,Typ_dnia,Pora_roku
Data czas,,,,,,,,
2016-01-01,4065.480609,-3.691304,3.260870,78.565217,7.304348,7.433,5.0,4.0
2016-01-02,4699.010000,-9.341667,4.416667,70.083333,1.416667,7.483,3.0,4.0
2016-01-03,5123.421500,-11.200000,3.708333,68.458333,0.041667,7.500,4.0,4.0
2016-01-04,5232.324750,-9.908333,3.750000,55.125000,0.000000,7.517,1.0,4.0
2016-01-05,5489.413208,-10.616667,1.916667,61.916667,1.333333,7.550,1.0,4.0


In [3]:
df_day_info = df[['Dlug_dnia', 'Typ_dnia', 'Pora_roku']].copy()
df_day_info.head()

,Dlug_dnia,Typ_dnia,Pora_roku
Data czas,,,
2016-01-01,7.433,5.0,4.0
2016-01-02,7.483,3.0,4.0
2016-01-03,7.500,4.0,4.0
2016-01-04,7.517,1.0,4.0
2016-01-05,7.550,1.0,4.0


In [4]:
df.drop(columns=['Dlug_dnia', 'Typ_dnia', 'Pora_roku'], inplace=True)
df.head()

,Energia,Temp_zewn,V_wiatru,Wilg,Zachm
Data czas,,,,,
2016-01-01,4065.480609,-3.691304,3.260870,78.565217,7.304348
2016-01-02,4699.010000,-9.341667,4.416667,70.083333,1.416667
2016-01-03,5123.421500,-11.200000,3.708333,68.458333,0.041667
2016-01-04,5232.324750,-9.908333,3.750000,55.125000,0.000000
2016-01-05,5489.413208,-10.616667,1.916667,61.916667,1.333333


In [5]:
pd_lag = series_to_supervised(df, 3)
pd_lag.head()

,Energia(t-3),Temp_zewn(t-3),V_wiatru(t-3),Wilg(t-3),Zachm(t-3),Energia(t-2),Temp_zewn(t-2),V_wiatru(t-2),Wilg(t-2),Zachm(t-2),Energia(t-1),Temp_zewn(t-1),V_wiatru(t-1),Wilg(t-1),Zachm(t-1),Energia(t),Temp_zewn(t),V_wiatru(t),Wilg(t),Zachm(t)
Data czas,,,,,,,,,,,,,,,,,,,,
2016-01-04,4065.480609,-3.691304,3.260870,78.565217,7.304348,4699.010000,-9.341667,4.416667,70.083333,1.416667,5123.421500,-11.200000,3.708333,68.458333,0.041667,5232.324750,-9.908333,3.750000,55.125000,0.000000
2016-01-05,4699.010000,-9.341667,4.416667,70.083333,1.416667,5123.421500,-11.200000,3.708333,68.458333,0.041667,5232.324750,-9.908333,3.750000,55.125000,0.000000,5489.413208,-10.616667,1.916667,61.916667,1.333333
2016-01-06,5123.421500,-11.200000,3.708333,68.458333,0.041667,5232.324750,-9.908333,3.750000,55.125000,0.000000,5489.413208,-10.616667,1.916667,61.916667,1.333333,5661.893417,-11.320833,1.583333,77.583333,0.125000
2016-01-07,5232.324750,-9.908333,3.750000,55.125000,0.000000,5489.413208,-10.616667,1.916667,61.916667,1.333333,5661.893417,-11.320833,1.583333,77.583333,0.125000,5415.852458,-7.220833,2.791667,69.541667,5.916667
2016-01-08,5489.413208,-10.616667,1.916667,61.916667,1.333333,5661.893417,-11.320833,1.583333,77.583333,0.125000,5415.852458,-7.220833,2.791667,69.541667,5.916667,4732.605458,-1.837500,3.916667,90.416667,7.958333


In [6]:
frames = [pd_lag, df_day_info]
df = pd.concat(frames, axis=1, join='inner')
df.head()

,Energia(t-3),Temp_zewn(t-3),V_wiatru(t-3),Wilg(t-3),Zachm(t-3),Energia(t-2),Temp_zewn(t-2),V_wiatru(t-2),Wilg(t-2),Zachm(t-2),...,Wilg(t-1),Zachm(t-1),Energia(t),Temp_zewn(t),V_wiatru(t),Wilg(t),Zachm(t),Dlug_dnia,Typ_dnia,Pora_roku
Data czas,,,,,,,,,,,,,,,,,,,,,
2016-01-04,4065.480609,-3.691304,3.260870,78.565217,7.304348,4699.010000,-9.341667,4.416667,70.083333,1.416667,...,68.458333,0.041667,5232.324750,-9.908333,3.750000,55.125000,0.000000,7.517,1.0,4.0
2016-01-05,4699.010000,-9.341667,4.416667,70.083333,1.416667,5123.421500,-11.200000,3.708333,68.458333,0.041667,...,55.125000,0.000000,5489.413208,-10.616667,1.916667,61.916667,1.333333,7.550,1.0,4.0
2016-01-06,5123.421500,-11.200000,3.708333,68.458333,0.041667,5232.324750,-9.908333,3.750000,55.125000,0.000000,...,61.916667,1.333333,5661.893417,-11.320833,1.583333,77.583333,0.125000,7.583,5.0,4.0
2016-01-07,5232.324750,-9.908333,3.750000,55.125000,0.000000,5489.413208,-10.616667,1.916667,61.916667,1.333333,...,77.583333,0.125000,5415.852458,-7.220833,2.791667,69.541667,5.916667,7.600,1.0,4.0
2016-01-08,5489.413208,-10.616667,1.916667,61.916667,1.333333,5661.893417,-11.320833,1.583333,77.583333,0.125000,...,69.541667,5.916667,4732.605458,-1.837500,3.916667,90.416667,7.958333,7.650,2.0,4.0


In [7]:
//df.to_csv('3dni_pogoda.csv')

SyntaxError: invalid syntax (<ipython-input-7-b3302e6ad75c>, line 1)